In [371]:
import pandas as pd
import numpy as np
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.preprocessing import LabelEncoder

In [372]:
total = pd.read_csv("data/tfidf_SVC_training.csv")

In [373]:
total.head(10)

,label,id,date,stuff,name,text
0,0,2284311891,Mon Jun 22 13:43:42 PDT 2009,NO_QUERY,kaysleecolleen,@ddlovato so sorry about your voice just know...
1,4,1831238993,Sun May 17 18:29:28 PDT 2009,NO_QUERY,vpennino,@ListPlanIt I prefer a nice pen with a thick p...
2,0,1694978230,Mon May 04 04:48:29 PDT 2009,NO_QUERY,theweirdwriter,"Gloomy day here I know that we need the rain,..."
3,4,2181702252,Mon Jun 15 12:10:56 PDT 2009,NO_QUERY,AndreMJonesJr,@Summa_Reign ooooh. well let's keep our finge...
4,0,2182272034,Mon Jun 15 12:57:10 PDT 2009,NO_QUERY,themaineisluvx3,It sucks when the guy you like likes someone e...
5,0,2050655071,Fri Jun 05 19:03:37 PDT 2009,NO_QUERY,amywong26,@richardtsang I was outside on the field of gr...
6,0,1978119622,Sat May 30 22:29:37 PDT 2009,NO_QUERY,nanchunja,Just finished ball. Facing the reality of assi...
7,4,2191518259,Tue Jun 16 05:26:36 PDT 2009,NO_QUERY,davidaguirre,'How can you live with yourself you horrible m...
8,4,1970545153,Sat May 30 04:17:39 PDT 2009,NO_QUERY,ohdesu,IM WITH PIGGGGY
9,4,2185464930,Mon Jun 15 17:25:20 PDT 2009,NO_QUERY,Doll_Doll,is watching Fringe. Josh Jackson will always b...


In [374]:
total.shape

(10000, 6)

In [375]:
data = total[["text", "label"]]

(0 = negative, 2 = neutral, 4 = positive)

In [376]:
polarity = {0: "negative", 2: "neutral", 4: "positive"}
data["sentiment"] = data["label"].apply(lambda x: polarity[x])
data["sentiment"].value_counts()

/tmp/ipykernel_96119/3603516048.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["sentiment"] = data["label"].apply(lambda x: polarity[x])


sentiment
positive    5065
negative    4935
Name: count, dtype: int64

In [377]:
polarity_change = {0: 0, 2: 2, 4: 1}

data["label"] = data["label"].apply(lambda x: polarity_change[x])

data.head(10)


/tmp/ipykernel_96119/3635157040.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["label"] = data["label"].apply(lambda x: polarity_change[x])


,text,label,sentiment
0,@ddlovato so sorry about your voice just know...,0,negative
1,@ListPlanIt I prefer a nice pen with a thick p...,1,positive
2,"Gloomy day here I know that we need the rain,...",0,negative
3,@Summa_Reign ooooh. well let's keep our finge...,1,positive
4,It sucks when the guy you like likes someone e...,0,negative
5,@richardtsang I was outside on the field of gr...,0,negative
6,Just finished ball. Facing the reality of assi...,0,negative
7,'How can you live with yourself you horrible m...,1,positive
8,IM WITH PIGGGGY,1,positive
9,is watching Fringe. Josh Jackson will always b...,1,positive


In [378]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       10000 non-null  object
 1   label      10000 non-null  int64 
 2   sentiment  10000 non-null  object
dtypes: int64(1), object(2)
memory usage: 234.5+ KB


In [379]:
# lowercasing
data.loc[:, "text_cleaned"] = data.loc[:, "text"].apply(lambda x: x.lower())

# remove @name tags
data.loc[:, "text_cleaned"] = data.loc[:, "text_cleaned"].apply(lambda x: re.sub(r"^@[\w]+", "", x))

# remove inet adress
data.loc[:, "text_cleaned"] = data.loc[:, "text_cleaned"].apply(lambda x: re.sub(r"https?://[\w\d/]+", "", x))

# removing punctioation
data.loc[:, "text_cleaned"] = data.loc[:, "text_cleaned"].apply(lambda x: re.sub(r"[^\w\s]", "", x).strip())

# remove numbers
data.loc[:, "text_cleaned"] = data.loc[:, "text_cleaned"].apply(lambda x: re.sub(r"[\d]", "", x).strip())

data.head(10)

/tmp/ipykernel_96119/445619657.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, "text_cleaned"] = data.loc[:, "text"].apply(lambda x: x.lower())


,text,label,sentiment,text_cleaned
0,@ddlovato so sorry about your voice just know...,0,negative,so sorry about your voice just know that your...
1,@ListPlanIt I prefer a nice pen with a thick p...,1,positive,i prefer a nice pen with a thick pad of paper
2,"Gloomy day here I know that we need the rain,...",0,negative,gloomy day here i know that we need the rain ...
3,@Summa_Reign ooooh. well let's keep our finge...,1,positive,ooooh well lets keep our fingers crossed then
4,It sucks when the guy you like likes someone e...,0,negative,it sucks when the guy you like likes someone e...
5,@richardtsang I was outside on the field of gr...,0,negative,i was outside on the field of grass my sis was...
6,Just finished ball. Facing the reality of assi...,0,negative,just finished ball facing the reality of assig...
7,'How can you live with yourself you horrible m...,1,positive,how can you live with yourself you horrible ma...
8,IM WITH PIGGGGY,1,positive,im with piggggy
9,is watching Fringe. Josh Jackson will always b...,1,positive,is watching fringe josh jackson will always be...


In [380]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


In [381]:
# en_stopwords = stopwords.words("english")
# data.loc[:, "text_cleaned"] = data.loc[:, "text_cleaned"].apply(
#     lambda x: " ".join([word for word in x.split(" ") if word not in en_stopwords])
# )

In [382]:
data.loc[:, "text_cleaned"] = data.loc[:, "text_cleaned"].apply(lambda x: word_tokenize(x))

In [383]:
data.head(10)

,text,label,sentiment,text_cleaned
0,@ddlovato so sorry about your voice just know...,0,negative,"[so, sorry, about, your, voice, just, know, th..."
1,@ListPlanIt I prefer a nice pen with a thick p...,1,positive,"[i, prefer, a, nice, pen, with, a, thick, pad,..."
2,"Gloomy day here I know that we need the rain,...",0,negative,"[gloomy, day, here, i, know, that, we, need, t..."
3,@Summa_Reign ooooh. well let's keep our finge...,1,positive,"[ooooh, well, lets, keep, our, fingers, crosse..."
4,It sucks when the guy you like likes someone e...,0,negative,"[it, sucks, when, the, guy, you, like, likes, ..."
5,@richardtsang I was outside on the field of gr...,0,negative,"[i, was, outside, on, the, field, of, grass, m..."
6,Just finished ball. Facing the reality of assi...,0,negative,"[just, finished, ball, facing, the, reality, o..."
7,'How can you live with yourself you horrible m...,1,positive,"[how, can, you, live, with, yourself, you, hor..."
8,IM WITH PIGGGGY,1,positive,"[im, with, piggggy]"
9,is watching Fringe. Josh Jackson will always b...,1,positive,"[is, watching, fringe, josh, jackson, will, al..."


In [384]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
data.loc[:, "text_cleaned"] = data.loc[:, "text_cleaned"].apply(
    lambda tokens: [lemmatizer.lemmatize(token) for token in tokens]
)


In [385]:
from nltk.util import ngrams

data["bigrams"] = data["text_cleaned"].apply(lambda tokens: list(ngrams(tokens, 2)))

data["trigrams"] = data["text_cleaned"].apply(lambda tokens: list(ngrams(tokens, 3)))


/tmp/ipykernel_96119/2469617569.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["bigrams"] = data["text_cleaned"].apply(lambda tokens: list(ngrams(tokens, 2)))


In [386]:
data.head(10)

,text,label,sentiment,text_cleaned,bigrams,trigrams
0,@ddlovato so sorry about your voice just know...,0,negative,"[so, sorry, about, your, voice, just, know, th...","[(so, sorry), (sorry, about), (about, your), (...","[(so, sorry, about), (sorry, about, your), (ab..."
1,@ListPlanIt I prefer a nice pen with a thick p...,1,positive,"[i, prefer, a, nice, pen, with, a, thick, pad,...","[(i, prefer), (prefer, a), (a, nice), (nice, p...","[(i, prefer, a), (prefer, a, nice), (a, nice, ..."
2,"Gloomy day here I know that we need the rain,...",0,negative,"[gloomy, day, here, i, know, that, we, need, t...","[(gloomy, day), (day, here), (here, i), (i, kn...","[(gloomy, day, here), (day, here, i), (here, i..."
3,@Summa_Reign ooooh. well let's keep our finge...,1,positive,"[ooooh, well, let, keep, our, finger, crossed,...","[(ooooh, well), (well, let), (let, keep), (kee...","[(ooooh, well, let), (well, let, keep), (let, ..."
4,It sucks when the guy you like likes someone e...,0,negative,"[it, suck, when, the, guy, you, like, like, so...","[(it, suck), (suck, when), (when, the), (the, ...","[(it, suck, when), (suck, when, the), (when, t..."
5,@richardtsang I was outside on the field of gr...,0,negative,"[i, wa, outside, on, the, field, of, grass, my...","[(i, wa), (wa, outside), (outside, on), (on, t...","[(i, wa, outside), (wa, outside, on), (outside..."
6,Just finished ball. Facing the reality of assi...,0,negative,"[just, finished, ball, facing, the, reality, o...","[(just, finished), (finished, ball), (ball, fa...","[(just, finished, ball), (finished, ball, faci..."
7,'How can you live with yourself you horrible m...,1,positive,"[how, can, you, live, with, yourself, you, hor...","[(how, can), (can, you), (you, live), (live, w...","[(how, can, you), (can, you, live), (you, live..."
8,IM WITH PIGGGGY,1,positive,"[im, with, piggggy]","[(im, with), (with, piggggy)]","[(im, with, piggggy)]"
9,is watching Fringe. Josh Jackson will always b...,1,positive,"[is, watching, fringe, josh, jackson, will, al...","[(is, watching), (watching, fringe), (fringe, ...","[(is, watching, fringe), (watching, fringe, jo..."


In [404]:
data["bigrams_str"] = data["bigrams"].apply(lambda grams: ["_".join(gram) for gram in grams])
data["trigrams_str"] = data["trigrams"].apply(lambda grams: ["_".join(gram) for gram in grams])


In [405]:
bigram_corpus = data["bigrams_str"].apply(lambda tokens: " ".join(tokens))
trigram_corpus = data["trigrams_str"].apply(lambda tokens: " ".join(tokens))

In [406]:
trigram_corpus

0       so_sorry_about sorry_about_your about_your_voi...
1       i_prefer_a prefer_a_nice a_nice_pen nice_pen_w...
2       gloomy_day_here day_here_i here_i_know i_know_...
3       ooooh_well_let well_let_keep let_keep_our keep...
4       it_suck_when suck_when_the when_the_guy the_gu...
                              ...                        
9995    watchinq_the_scorpion the_scorpion_king scorpi...
9996    currently_a_big a_big_fan big_fan_of fan_of_sl...
9997    wow_i_wa i_wa_texting wa_texting_and texting_a...
9998    im_so_craving so_craving_coffee craving_coffee...
9999                                                     
Name: trigrams_str, Length: 10000, dtype: object

In [407]:
Y = data["label"]

In [408]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

countvec = CountVectorizer()

X = countvec.fit_transform(bigram_corpus)


TF-IDF Logistic Regression

In [413]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer()
X = tfidf.fit_transform(X)


In [414]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.15)


In [415]:
from sklearn.model_selection import GridSearchCV

param_grid = {"C": [0.01, 0.1, 1, 10]}
lr = LogisticRegression(class_weight="balanced", max_iter=1000)

grid = GridSearchCV(estimator=lr, param_grid=param_grid, cv=5)
grid.fit(x_train, y_train)
print(grid.best_score_)
print(grid.best_estimator_)

0.6050588235294118
LogisticRegression(C=1, class_weight='balanced', max_iter=1000)


In [416]:
lr = LogisticRegression(class_weight="balanced", max_iter=1000, C=1)
lr.fit(x_train, y_train)
y_pred = lr.predict(x_test)
acc = accuracy_score(y_test, y_pred)


In [417]:
print(acc)

0.602


TF-IDF Naive Bayes

In [418]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB().fit(x_train, y_train)
y_pred = model.predict(x_test)
acc = accuracy_score(y_test, y_pred)
print(acc)


0.606


TF-IDF SVM 

In [419]:
from sklearn.svm import LinearSVC

svc = LinearSVC(class_weight="balanced")
params = {"C": [0.01, 0.1, 1, 10]}
grid = GridSearchCV(estimator=svc, param_grid=param_grid, cv=5)
grid.fit(x_train, y_train)
print(grid.best_score_)
print(grid.best_estimator_)


0.6050588235294118
LinearSVC(C=0.01, class_weight='balanced')


In [420]:
model = LinearSVC(**grid.best_params_).fit(x_train, y_train)
y_pred = model.predict(x_test)
acc = accuracy_score(y_test, y_pred)
print(acc)


0.5193333333333333


In [354]:
import joblib

joblib.dump(countvec, "ML_models/CountVec.pkl")
joblib.dump(model, "ML_models/SVC.pkl")
joblib.dump(tfidf, "ML_models/tfidf_transformer.pkl")

['ML_models/tfidf_transformer.pkl']

In [228]:
print(x_test.toarray())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
